https://www.youtube.com/watch?v=06TE_U21FK4

GET THE CODE FROM THE VIDEO:
https://github.com/nicknochnack/MediaPipePoseEstimation

# 1. Install and Import Dependencies

In [1]:
!pip install mediapipe opencv-python

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 使用影片檔案來源
cap = cv2.VideoCapture("/Users/peilinyang/Desktop/Leading_leg/4.mov")

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.9) as pose:
    while cap.isOpened():
        # 讀取影格
        ret, frame = cap.read()
        
        # 檢查是否成功讀取到影格
        if not ret:
            print("End of video. Exiting...")
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        # 顯示處理後的影像
        cv2.imshow('Mediapipe Feed', image)

        # 按 'q' 鍵退出
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # 釋放影片檔案來源
    cap.release()
    cv2.destroyAllWindows()

2023-12-12 10:20:44.878184: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[x: 0.5000008344650269
y: 0.14772191643714905
z: -0.4573914408683777
visibility: 0.9999964237213135
, x: 0.5085585713386536
y: 0.13363486528396606
z: -0.440509557723999
visibility: 0.999983549118042
, x: 0.5155243873596191
y: 0.1334509253501892
z: -0.44042402505874634
visibility: 0.9999853372573853
, x: 0.5219833850860596
y: 0.1334557831287384
z: -0.44042137265205383
visibility: 0.9999849796295166
, x: 0.4870818853378296
y: 0.13594573736190796
z: -0.4420470595359802
visibility: 0.9999840259552002
, x: 0.4794587194919586
y: 0.13732174038887024
z: -0.4420315623283386
visibility: 0.9999850988388062
, x: 0.4722348749637604
y: 0.13871508836746216
z: -0.44187626242637634
visibility: 0.999982476234436
, x: 0.5304030776023865
y: 0.1441037952899933
z: -0.30311140418052673
visibility: 0.999967098236084
, x: 0.458515465259552
y: 0.15040063858032227
z: -0.31302759051322937
visibility: 0.9999723434448242
, x: 0.5160728693008423
y: 0.1696627140045166
z: -0.4027893543243408
visibility: 0.999994158744

In [3]:
landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

x: 0.451372891664505
y: 0.500812828540802
z: 0.003314842702820897
visibility: 0.9996097683906555

In [4]:
landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]

x: 0.45219674706459045
y: 0.711712121963501
z: -0.011981186456978321
visibility: 0.9742166996002197

In [5]:
landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]

x: 0.47037816047668457
y: 0.9159888625144958
z: 0.125594362616539
visibility: 0.9524116516113281

In [6]:
landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

x: 0.5669622421264648
y: 0.4974021315574646
z: -0.003903885604813695
visibility: 0.9996171593666077

In [7]:
landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]

x: 0.5457862019538879
y: 0.7203789353370667
z: 0.005977335385978222
visibility: 0.9766836762428284

In [8]:
landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

x: 0.5403160452842712
y: 0.9011226296424866
z: 0.14489319920539856
visibility: 0.9754041433334351

# 3. Calculate Angles

Leading leg 的關鍵點是 右側：24, 26, 28/ 左側：23, 25,27

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [9]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.9)

# Video capture from file
cap = cv2.VideoCapture('/Users/peilinyang/Desktop/Leading_leg/4.mov')
# 即時影像偵測：cap = cv2.VideoCapture(0)

# Knee counter variables
left_counter = 0
right_counter = 0
left_stage = None
right_stage = None

## Setup mediapipe instance
while cap.isOpened():
    # 讀取影格
    ret, frame = cap.read()

    # 檢查是否成功讀取到影格
    if not ret:
        print("End of video. Exiting...")
        break

    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Make detection
    results = pose.process(image)

    # Recolor back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    try:
        landmarks = results.pose_landmarks.landmark

        # Extract landmarks for the left leg
        left_hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
        left_knee = np.array([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y])
        left_ankle = np.array([landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])

        # Extract landmarks for the right leg
        right_hip = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y])
        right_knee = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y])
        right_ankle = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y])

        # Calculate angle for the left leg
        vector1_left = left_hip - left_knee
        vector2_left = left_ankle - left_knee
        dot_product_left = np.dot(vector1_left, vector2_left)
        norm_product_left = np.linalg.norm(vector1_left) * np.linalg.norm(vector2_left)
        cos_theta_left = dot_product_left / norm_product_left
        angle_rad_left = np.arccos(np.clip(cos_theta_left, -1.0, 1.0))
        angle_deg_left = np.degrees(angle_rad_left)

        # Calculate angle for the right leg
        vector1_right = right_hip - right_knee
        vector2_right = right_ankle - right_knee
        dot_product_right = np.dot(vector1_right, vector2_right)
        norm_product_right = np.linalg.norm(vector1_right) * np.linalg.norm(vector2_right)
        cos_theta_right = dot_product_right / norm_product_right
        angle_rad_right = np.arccos(np.clip(cos_theta_right, -1.0, 1.0))
        angle_deg_right = np.degrees(angle_rad_right)

        # Visualize angle for the left leg
        cv2.putText(image, f"Left Leg Angle: {angle_deg_left:.2f}", tuple(np.multiply(left_knee, [1400, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (245, 66, 230), 2, cv2.LINE_AA)

        # Visualize angle for the right leg
        cv2.putText(image, f"Right Leg Angle: {angle_deg_right:.2f}", tuple(np.multiply(right_knee, [480, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)


        # Check the left leg angle 
        if angle_deg_left is not None and angle_deg_left > 170:
            left_stage = "down"
        if angle_deg_left is not None and angle_deg_left < 90 and left_stage == 'down':
            left_stage = "up"
            left_counter += 1
            print(left_counter)

        # Check the right leg angle 
        if angle_deg_right is not None and angle_deg_right > 170:
            right_stage = "down"
        if angle_deg_right is not None and angle_deg_right < 90 and right_stage == 'down':
            right_stage = "up"
            right_counter += 1
            print(right_counter)

    except:
        pass

    # Render knee counter
    # Setup status box
    cv2.rectangle(image, (0, 0), (460, 100), (245, 117, 16), -1)

    # Rep data for left leg
    cv2.putText(image, 'REPS Right', (15, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(right_counter),
                (15, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2, cv2.LINE_AA)

    # Stage data for left leg
    cv2.putText(image, 'RIGHT STAGE', (125, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, right_stage,
                (125, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Rep data for right leg
    cv2.putText(image, 'REPS Left', (250, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(left_counter),
                (250, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (245,66,230), 2, cv2.LINE_AA)

    # Stage data for left leg
    cv2.putText(image, 'LEFT STAGE', (350, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, left_stage,
                (350, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Render detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

    cv2.imshow('Mediapipe Feed', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Output final counters
print(f"Final Left leg Reps: {left_counter}")
print(f"Final Right leg Reps: {right_counter}")

1
1
2
2
End of video. Exiting...
Final Left leg Reps: 2
Final Right leg Reps: 2


In [10]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.9)

# Video capture from file
cap = cv2.VideoCapture('/Users/peilinyang/Desktop/Leading_leg/3.mov')
# 即時影像偵測：cap = cv2.VideoCapture(0)

# Knee counter variables
left_counter = 0
right_counter = 0
left_stage = None
right_stage = None

## Setup mediapipe instance
while cap.isOpened():
    # 讀取影格
    ret, frame = cap.read()

    # 檢查是否成功讀取到影格
    if not ret:
        print("End of video. Exiting...")
        break

    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Make detection
    results = pose.process(image)

    # Recolor back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    try:
        landmarks = results.pose_landmarks.landmark

        # Extract landmarks for the left leg
        left_hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
        left_knee = np.array([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y])
        left_ankle = np.array([landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])

        # Extract landmarks for the right leg
        right_hip = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y])
        right_knee = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y])
        right_ankle = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y])

        # Calculate angle for the left leg
        vector1_left = left_hip - left_knee
        vector2_left = left_ankle - left_knee
        dot_product_left = np.dot(vector1_left, vector2_left)
        norm_product_left = np.linalg.norm(vector1_left) * np.linalg.norm(vector2_left)
        cos_theta_left = dot_product_left / norm_product_left
        angle_rad_left = np.arccos(np.clip(cos_theta_left, -1.0, 1.0))
        angle_deg_left = np.degrees(angle_rad_left)

        # Calculate angle for the right leg
        vector1_right = right_hip - right_knee
        vector2_right = right_ankle - right_knee
        dot_product_right = np.dot(vector1_right, vector2_right)
        norm_product_right = np.linalg.norm(vector1_right) * np.linalg.norm(vector2_right)
        cos_theta_right = dot_product_right / norm_product_right
        angle_rad_right = np.arccos(np.clip(cos_theta_right, -1.0, 1.0))
        angle_deg_right = np.degrees(angle_rad_right)

        # Visualize angle for the left leg
        cv2.putText(image, f"Left Leg Angle: {angle_deg_left:.2f}", tuple(np.multiply(left_knee, [1400, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (245, 66, 230), 2, cv2.LINE_AA)

        # Visualize angle for the right leg
        cv2.putText(image, f"Right Leg Angle: {angle_deg_right:.2f}", tuple(np.multiply(right_knee, [480, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)


        # Check the left leg angle 
        if angle_deg_left is not None and angle_deg_left > 170:
            left_stage = "down"
        if angle_deg_left is not None and angle_deg_left < 90 and left_stage == 'down':
            left_stage = "up"
            left_counter += 1
            print(left_counter)

        # Check the right leg angle 
        if angle_deg_right is not None and angle_deg_right > 170:
            right_stage = "down"
        if angle_deg_right is not None and angle_deg_right < 90 and right_stage == 'down':
            right_stage = "up"
            right_counter += 1
            print(right_counter)

    except:
        pass

    # Render knee counter
    # Setup status box
    cv2.rectangle(image, (0, 0), (460, 100), (245, 117, 16), -1)

    # Rep data for left leg
    cv2.putText(image, 'REPS Right', (15, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(right_counter),
                (15, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2, cv2.LINE_AA)

    # Stage data for left leg
    cv2.putText(image, 'RIGHT STAGE', (125, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, right_stage,
                (125, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Rep data for right leg
    cv2.putText(image, 'REPS Left', (250, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, str(left_counter),
                (250, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (245,66,230), 2, cv2.LINE_AA)

    # Stage data for left leg
    cv2.putText(image, 'LEFT STAGE', (350, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, left_stage,
                (350, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Render detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

    cv2.imshow('Mediapipe Feed', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Output final counters
print(f"Final Left leg Reps: {left_counter}")
print(f"Final Right leg Reps: {right_counter}")

1
1
2
2
End of video. Exiting...
Final Left leg Reps: 2
Final Right leg Reps: 2
